## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [3]:
import pandas as pd
pr = pd.read_csv('/datasets/data.csv')
pr.info()
pr.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Данная таблица состоит из 12 столбцов и 21525 строк, которые имеют следующие типы данных: int64, float64 и object. А в столбцах days_employed и total_income присутствует одинаковое количество пропусков.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#print(pr[pr['days_employed'].isnull()].count())
pr.loc[pr['children'] == 20, 'children'] = 2
pr.loc[pr['children'] < 0, 'children'] = - pr['children']
pr.loc[pr['days_employed'] < 0, 'days_employed'] = - pr['days_employed'] #заменили все значения на положительные 
pr['days_employed'] = pr['days_employed'].fillna(pr.groupby('income_type')['days_employed'].transform('mean')) #заменяем все NaN в 'days_employed' на средние значения для пенсионеров, сотрудников и т.д.
pr['total_income'] = pr['total_income'].fillna(pr.groupby('income_type')['total_income'].transform('mean')) #заменяем все NaN в 'total_income' на средние значения для пенсионеров, сотрудников и т.д.
pr.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Мы имеем соотвествующие пропуски в столбцах days_employed и total_income, у данных пропусков нет никакой связи с другими столбцами, поэтому принято решение заменить все NaN на средние значения по квалификации людей соотвественно. Причина возникновения, скорее всего, является техническая ошибка из-за данных на вводе. Также замечены ошибки в данных в столбце 'children', детей не может быть -1, а также 76 человек имело 20 детей, странное совпадение, скорее всего это оппечатка и на самом деле у них 2 детей.

### Замена типа данных

In [5]:
pr['days_employed'] = pr['days_employed'].astype('int')
pr['total_income'] = pr['total_income'].astype('int')
pr['children'] = pr['children'].astype('int')
pr.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Я поменял тип данных у двух столбцов days_employed, total_income , т.к. для подсчета количества дней корректно использовать целочисленные значения, а для дохода не так критично потерять значения после запятой.

### Обработка дубликатов

In [6]:
#pr.duplicated().sum()
pr['education'] = pr['education'].str.lower()
#pr.duplicated().sum()
pr.drop_duplicates().reset_index(drop = True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Я искал дубликаты с учетом регистра, их оказалось 71, после чего я их удалил. Скорее всего, причиной возникновения дубликатов является человеческий фактор, т.к. их количество мало и некоторые связаны с регистром.

### Лемматизация

In [7]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 
#print(Counter(pr_purpose))
#print(Counter(m.lemmatize(pr_purpose))

purpose_list = str(pr['purpose'].unique()) 
m = Mystem()
pr['purpose_lemmas'] = pr['purpose'].apply(lambda x: [(' '.join(m.lemmatize(x)))])
def listToString(ddd):  
    str1 = ""  
    for kkk in ddd:  
        str1 += kkk   
    
     
    return str1  
pr['purpose_lemmas'] = pr['purpose_lemmas'].apply(listToString)

def lemm(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'жилье' in row:
        return 'жилье'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'
    if 'автомобиль' in row:
        return 'автомобиль'
    else:
        return 0
pr['purpose_lemmas'] = pr['purpose_lemmas'].apply(lambda x: lemm(x))
pr.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


### Вывод

Можно заметить, что самая расспространенная цель это покупка, а также 5 основных объектов, над которыми осуществляется та или иная операция: недвижимость, автомобиль, образование, жилье, свадьба.

### Категоризация данных

In [9]:
#print(pr['children'].value_counts())
def zzzz(ssss):
    if ssss == 0:
        return 'без детей'
    if ssss > 0: 
        return 'есть дети'
pr['have_child'] = pr['children'].apply(zzzz)
pr_child = pr[['have_child', 'debt']]
#print(pr_child)
pr_family = pr[['family_status', 'debt']]
def uuuu(aaaa):
    if aaaa <= 100000:
        return 'низкий'
    if aaaa > 100000 and aaaa < 200000:
        return 'средний'
    if aaaa >= 200000:
        return 'высокий'
pr['type_income'] = pr['total_income'].apply(uuuu)
pr_income = pr[['type_income', 'debt']]
pr_pur = pr[['purpose_lemmas', 'debt']]
pr.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,have_child,type_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,есть дети,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,без детей,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,без детей,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,без детей,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,без детей,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,без детей,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,есть дети,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,без детей,средний


### Вывод

Данные категоризированы по наличию детей и возврата кредита срок, по семейному статусу и возврата кредита в срок, по типу доходности и возврата кредита срок и соотвественно по причине кредита и возврата кредита в срок, вы можете заметить вышеописанные 'словари' для легкого и корректного ответа на нижеописанные вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
print(pr['have_child'].value_counts())
print(pr_child.groupby('have_child').sum())
df = pd.DataFrame({'Всего': [14149, 7376], 'Должники': [1063, 678]})
df['Относительный показатель должников от всей группы'] = df['Должники']/df['Всего']
print(df)

без детей    14149
есть дети     7376
Name: have_child, dtype: int64
            debt
have_child      
без детей   1063
есть дети    678
   Всего  Должники  Относительный показатель должников от всей группы
0  14149      1063                                           0.075129
1   7376       678                                           0.091920


In [12]:
pr_child.groupby('have_child').agg({'debt': ['count', 'sum', 'mean']}).style.format({('debt', 'mean'): '{:.2%}'})

### Вывод

Зависимости особой нет, т.к. процент должников в обеих группах почти равна.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
print(pr['family_status'].value_counts())
print(pr_family.groupby('family_status').sum())
df1 = pd.DataFrame({'Всего': [12380, 4177, 2813, 1195, 960], 'Должники': [931, 388, 274, 85, 63]})
df1['Относительный показатель должников от всей группы'] = df1['Должники']/df1['Всего']
print(df1)

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
                       debt
family_status              
Не женат / не замужем   274
в разводе                85
вдовец / вдова           63
гражданский брак        388
женат / замужем         931
   Всего  Должники  Относительный показатель должников от всей группы
0  12380       931                                           0.075202
1   4177       388                                           0.092890
2   2813       274                                           0.097405
3   1195        85                                           0.071130
4    960        63                                           0.065625


### Вывод

Процент должников  со статусами 'Не женат / не замужем' и 'гражданский брак', чуть выше чем у остальных

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
print(pr['type_income'].value_counts())
print(pr_income.groupby('type_income').sum())
df2 = pd.DataFrame({'Всего': [5575, 11487, 4463], 'Должники': [388, 999, 354]})
df2['Относительный показатель должников от всей группы'] = df2['Должники']/df2['Всего']
print(df2) #Не совсем разобрался в способе, который ты предложил, поэтому решил сделать вот так.

средний    11487
высокий     5575
низкий      4463
Name: type_income, dtype: int64
             debt
type_income      
высокий       388
низкий        354
средний       999
   Всего  Должники  Относительный показатель должников от всей группы
0   5575       388                                           0.069596
1  11487       999                                           0.086968
2   4463       354                                           0.079319


### Вывод

Особой зависимости не наблюдается, самый высокий процент у среднего дохода, а самый низкий у высокого дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
print(pr['purpose_lemmas'].value_counts())
print(pr_pur.groupby('purpose_lemmas').sum())
df3 = pd.DataFrame({'Всего': [6367, 4473, 4315, 4022, 2348], 'Должники': [474, 308, 403, 370, 186]})
df3['Относительный показатель должников от всей группы'] = df1['Должники']/df1['Всего']
print(df3)

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_lemmas, dtype: int64
                debt
purpose_lemmas      
автомобиль       403
жилье            308
недвижимость     474
образование      370
свадьба          186
   Всего  Должники  Относительный показатель должников от всей группы
0   6367       474                                           0.075202
1   4473       308                                           0.092890
2   4315       403                                           0.097405
3   4022       370                                           0.071130
4   2348       186                                           0.065625


### Вывод

Также не наблюдается особой зависимости, но можно заметить, что чуть больше людей имели задолжность с целью автомобиль и жилье, у остальных разница около 1%.

### Шаг 4. Общий вывод

В начале работы в таблице были замечены множество ошибок по причине технических неполадок, а также из-за простой человеческой невнимательности. Далее, после анализа таблицы, многие данные были восстановлены, также исправлен тип данных у некоторых столбцов на более коректный, а также была проведена лемматизация и категоризация, которые в последствии даст нам возможность без особых проблем проанализировать зависимость наших переменных. Переходя непосредственно к нашему анализу, можно сделать следующий вывод, если рассматривать риски задолжности со стороны клиента, стоит первоначально обратить внимание на его семейное положение, т.к. там самый большой разброс. 